In [ ]:
#Identification of novel cells

In [ ]:
import os
import random
import numpy as np
import yaml

import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler

from dataset import Cell_Dataset
from model import AutoEncoder
import sys
import multiprocessing

if __name__ == '__main__':
    multiprocessing.freeze_support()
    # 设置随机种子
    def setup_seed(seed):
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True

    # 训练参数
    #------------
    data_name = 'Muraro' 
    #------------
    
    train_path = f'D:\\Jupyter\\2C5221\\data\\{data_name}\\selected_train.csv'
    cfg_path = f'D:\\Jupyter\\2C5221\data\\{data_name}\\{data_name}.yaml'

    with open(cfg_path, 'rb') as f:
        cfg = yaml.load(f, yaml.SafeLoader)

    setup_seed(cfg['seed'])  # 随机种子
    batch_size = cfg['batch_size']  # 训练批次
    epochs = cfg['epochs']  # 训练步数

    # 自编码模型
    model = AutoEncoder()

    # 损失函数
    criterion = nn.MSELoss()

    # 优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # 混合精度训练
    scaler = GradScaler()

    # 学习率衰减
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-4)

    # 载入训练数据集
    train_dataset = Cell_Dataset(train_path)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)

    # 训练
    for epoch in range(1, epochs):
        loss_all = 0
        model.train()
        for i, data in enumerate(train_loader):
            data = data.float()

            with autocast():
                # 前向传播
                input_handle, output = model(data)

                # 计算损失函数
                loss = criterion(output, input_handle)

            loss_all += loss.item() / len(train_dataset) * batch_size

            # 反向传播
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        # 学习率更新
        scheduler.step()

        # 打印训练信息
        print('Epoch: [{}]\t train_loss: {:.4f}\t'.format(epoch, loss_all))

        # model save
        checkpoint = './data/{}/{}.pt'.format(data_name, data_name)
        torch.save(model.state_dict(), checkpoint)

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from model import AutoEncoder
from dataset import Cell_Dataset
import numpy as np
import sys
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import multiprocessing

if __name__ == '__main__':
    multiprocessing.freeze_support()

    # 加载模型
    def model_load(weight_path):
        checkpoint = torch.load(weight_path, map_location= 'cpu')
        model = AutoEncoder()
        model.load_state_dict(checkpoint)
        model.eval()

        criterion = nn.MSELoss()

        return model, criterion

    # 数据加载器
    def create_dataloader(data_path):
        dataset = Cell_Dataset(data_path)
        dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=True)
        return dataloader

    # 计算LOSS
    def compute_loss(dataloader, model, criterion):
        loss_list = []
        for i, data in enumerate(dataloader):
            data = data.float()
            input_handle, output = model(data)
            # compute mse_loss
            loss = criterion(output, input_handle)
            loss_list.append(loss.item())
        return loss_list

    # LOSS分布图
    def plot_loss(loss_list, save_name):
        loss_list = [round(i, 3) for i in loss_list]
        counter = sorted(Counter(loss_list).items())
        x_list = []
        y_list = []
        for x, y in counter:
            x_list.append(x)
            y_list.append(y)

        fig, ax = plt.subplots()
        sns.barplot(x=x_list, y=y_list, ax=ax)
        plt.xticks(rotation=45)

        ax.set(xlabel='mse_loss', ylabel='density')
        plt.tight_layout()
        
        plt.savefig(save_name, format='eps')
        plt.show(ax)
        
    def test(weight_path, data_path, save_name):
        model, criterion = model_load(weight_path)
        dataloader = create_dataloader(data_path)
        loss_list = compute_loss(dataloader, model, criterion)
        plot_loss(loss_list, save_name)

    #------------------
    data_name = 'Muraro' 
    #------------------
    
    weight = f'D:\\Jupyter\\2C5221\\data\\{data_name}\\{data_name}.pt'
    train_path = f'D:\\Jupyter\\2C5221\\data\\{data_name}\\selected_train.csv'
    # test_path = f'data/{data_type}/selected_test.csv'


    test(weight, train_path, f"D:\\Jupyter\\2C5221\\data\\{data_name}/train_loss.eps")

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from model import AutoEncoder
from dataset import Cell_Dataset
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import yaml
import sys
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(threshold = np.inf) 
import multiprocessing

if __name__ == '__main__':
    multiprocessing.freeze_support()
    
    #----------------
    data_name = 'Muraro2' # 可选：Muraro, 10x_5cl, Segerstolpe
    #---------------
    
    # 模型路径
    weight = f'D:\\Jupyter\\2C5221\\data\\{data_name}\\{data_name}.pt'

    # 测试集路径
    test_path = f'D:\\Jupyter\\2C5221\\data\\{data_name}\\selected_test.csv'
    label_path = f'D:\\Jupyter\\2C5221\\data\\{data_name}\\x_test_L.csv'

    # 超参数路径
    cfg_path = f'data/{data_name}/{data_name}.yaml'

    with open(cfg_path, 'rb') as f:
        cfg = yaml.load(f, yaml.SafeLoader)

    anomaly_threshold = cfg['anomaly_threshold'] # 异常阈值

    # 计算正确率
    def calculate_accuracy(y_true, y_pred):
        accuracy = accuracy_score(y_true, y_pred)
        return accuracy

    # 模型加载
    def model_load(weight_path):
        checkpoint = torch.load(weight_path, map_location= 'cpu')
        model = AutoEncoder()
        model.load_state_dict(checkpoint)
        model.eval()
        return model

    # 加载测试集数据和标签
    test_label = pd.read_csv(label_path)
    test_label = test_label.values.tolist()

    label_list = []
    for i in test_label:
        label_list.append(i[1])

    label = np.array(label_list)
    print("label:", label)

    # 加载模型
    model =  model_load(weight)
    criterion = nn.MSELoss()

    # 数据加载器
    test_dataset = Cell_Dataset(test_path)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=True)

    # 预测
    predict_list = []
    for i, data in enumerate(test_loader):
        data = data.float()
        input_handle, output = model(data)
        # compute mse_loss
        loss = criterion(output, input_handle)

        # 根据重构阈值预测已知细胞和未知细胞
        if loss.item() > anomaly_threshold:
            predict_list.append(1)
        else:
            predict_list.append(0)

    predict = np.array(predict_list)
    print("predict:", predict)

    # acc
    distinguish accuracy = calculate_accuracy(label, predict)
    
    #保存结果
    output_file = './data/Muraro2/predictions.csv'
    df = pd.DataFrame({'predict': predict})
    df.to_csv(output_file, index=False)

    print("distinguish accuracy:{}".format(acc, '.4f') )

In [ ]:
#Annotation of overall cells

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, adjusted_rand_score

In [ ]:
# 读取训练集数据和标签
train_data = pd.read_csv('XX/selected_train.csv')
train_labels = pd.read_csv('XX/x_train_L.csv', usecols=['x'])

# 计算标准差并按降序排序
std = train_data.std()
std_sorted = std.sort_values(ascending=False)

# 选择前n_feature_svm个特征索引
n_feature_svm = 1000
selected_features = std_sorted.index[:n_feature_svm]

# 提取选定的特征列
train_features = train_data[selected_features]

# 标签编码
label_encoder = LabelEncoder()
train_encoded_labels = label_encoder.fit_transform(train_labels.values.ravel())

# 训练逻辑回归模型
logistic_model = LogisticRegression()
logistic_model.fit(train_features, train_encoded_labels)

# 读取测试集数据
test_data = pd.read_csv('XX/selected_test.csv')

# 提取选定的特征列
test_features = test_data[selected_features]

# 预测测试集标签
test_predicted_labels = logistic_model.predict(test_features)

# 将预测结果映射回原始标签编码
test_predicted_labels = label_encoder.inverse_transform(test_predicted_labels)

# 将预测结果保存为CSV文件
test_predicted_labels = pd.Series(test_predicted_labels)

# 读取标签文件
labels = pd.read_csv('XX/predictions.csv')

# 记住值为1的索引
unknown_indices = labels[labels['predict'] == 1].index

# 在test_predicted_labels中将对应索引的行的标签改为"Unknown"
test_predicted_labels.loc[unknown_indices] = "Unknown"
test_predicted_labels.to_csv('XX/logistic_updated_predictions.csv', index=False)

real_labels = pd.read_csv('XX/x_test_L.csv', usecols=['z'])

# 计算准确率
full accuracy = accuracy_score(test_predicted_labels, real_labels)
print("准确率：", full accuracy)